# Sudoku Project
by Finn Potason (s2.wang@vu.nl)

This is a quick tutorial for MSc Logic students and those who struggle to kick off the project.

First of all you need the Pycosat solver [1].
In your terminal [2], type: pip install pycosat

[1] See more details at: https://pypi.python.org/pypi/pycosat
[2] Press ctrl + alt + t.

Next we test if the solver is working: 

In [10]:
import pycosat
import numpy as np
from math import sqrt
cnf = [[1, 5,4], [-1,5,3,4], [-3, -4]]
pycosat.solve(cnf)


[1, -2, -3, -4, 5]

Next, you can print all the possible assignments to this CNF. 

In [2]:
print ('There are in total ', len(list(pycosat.itersolve(cnf))), ' possible assignments:')
for sol in pycosat.itersolve(cnf):
    print(sol)


There are in total  18  possible assignments:
[1, -2, -3, -4, 5]
[1, -2, -3, 4, -5]
[1, -2, -3, 4, 5]
[1, -2, 3, -4, -5]
[1, -2, 3, -4, 5]
[1, 2, 3, -4, -5]
[1, 2, 3, -4, 5]
[1, 2, -3, -4, 5]
[1, 2, -3, 4, -5]
[1, 2, -3, 4, 5]
[-1, 2, -3, 4, -5]
[-1, 2, -3, 4, 5]
[-1, 2, -3, -4, 5]
[-1, 2, 3, -4, 5]
[-1, -2, 3, -4, 5]
[-1, -2, -3, -4, 5]
[-1, -2, -3, 4, -5]
[-1, -2, -3, 4, 5]


## Encode Sudoku
Take a random sudoku and think what the propositional variables are.
PS: You will need the library numpy. If you don't have it, install it (and reboot if needed).

In [3]:
import numpy as np
s_test = np.array([[8,0,6,5,0,0,0,0,0],
                    [0,0,4,0,0,0,0,0,8],
                    [0,0,0,0,0,0,6,0,0],
                    [0,0,0,0,0,0,0,0,0],
                    [3,7,0,4,5,0,0,0,0],
                    [5,0,1,0,9,8,0,0,7],
                    [0,0,0,0,0,7,0,2,0],
                    [2,5,7,1,6,0,0,0,9],
                    [0,8,0,0,3,0,0,4,0]])
print(s_test)

[[8 0 6 5 0 0 0 0 0]
 [0 0 4 0 0 0 0 0 8]
 [0 0 0 0 0 0 6 0 0]
 [0 0 0 0 0 0 0 0 0]
 [3 7 0 4 5 0 0 0 0]
 [5 0 1 0 9 8 0 0 7]
 [0 0 0 0 0 7 0 2 0]
 [2 5 7 1 6 0 0 0 9]
 [0 8 0 0 3 0 0 4 0]]


In [11]:
n = 9

names = np.zeros([n,n,n], dtype = np.int)
ids = list()
ids.append((-1,-1,-1))
index = 1
for i in range(n):
    for j in range(n):
        for k in range(n):
            name = i * n**2 + j * n + k+1
            names[i][j][k] = name
            
            ids.append((i+1,j+1,k+1))
#             print('if the cell at row: ', i+1, ' column: ', j+1, 'is ', k+1, 'then it is named ', names[i][j][k])

In [12]:
def get_name(names, i, j, k):
    return names[i-1, j-1, k-1]

print(get_name(names, 2,1,3))
print(ids[12])

84
(1, 2, 3)


In [6]:
# print ('When the cell at row 3 column 4 is 6')
# print('the variable' , names[2][3][5], ' is true')
# print('please be careful about the indexing!!!')

In [7]:
encode_cnf = []
print('start populate this cnf:', encode_cnf)

start populate this cnf: []


Next, we define two functions to obtain the function exactly one. 

In [13]:
def encode_at_most_one(names):
    encode = []
    for i in range(n):
        for j in range(n):
            for k in range(n-1):
                for l in range(k+1, n):
                    arr = [int(-names[i,j,k]), int(-names[i,j,l])]
                    encode.append(arr)
    return encode

def encode_at_least_one(names):
    encode = []
    for i in range(n):
        for j in range(n):
            arr = [int(names[i][j][k]) for k in range(n)]
            encode.append(arr)
    return encode


def index_finder(enc):
    for clause in enc:
        cl = ""
        for literal in clause:   
            if literal < 0:
                cl = cl + "~" + str(ids[-literal])
            else:
                cl = cl + str(ids[literal])
        print(cl)


In [9]:


pycosat.itersolve(exactly_one(names))

NameError: name 'exactly_one' is not defined

In [ ]:
for i in [721, 722, 723, 724, 725, 726, 727, 728, 729]:
    print(ids[i])

## 1) For each cell, exactly one variable is true.

In [14]:
def exactly_one(names):
    enc1 = encode_at_most_one(names)
    enc2 = encode_at_least_one(names)
    return enc1 + enc2

enc = exactly_one(names)
# index_finder(enc)

## 2) For each column, for each number, exactly one propositional variable is true.

In [15]:
def atleast_column(names):
    encode = []
    for j in range(n):
        for k in range(n):
            arr = [int(names[i][j][k]) for i in range(n)]
            encode.append(arr)
    return encode

def atmost_column(names):
    encode = []
    for k in range(n):
        for l in range(n):
            for i in range(n-1):
                for j in range(i+1, n):
                    arr = [int(-names[i,k,l]), int(-names[j,k,l])]
                    encode.append(arr)
    return encode

def exactly_one_column(names):
    enc1 = atmost_column(names)
    enc2 = atleast_column(names)
    return enc1 + enc2

index_finder(exactly_one_column(names))

~(1, 1, 1)~(2, 1, 1)
~(1, 1, 1)~(3, 1, 1)
~(1, 1, 1)~(4, 1, 1)
~(1, 1, 1)~(5, 1, 1)
~(1, 1, 1)~(6, 1, 1)
~(1, 1, 1)~(7, 1, 1)
~(1, 1, 1)~(8, 1, 1)
~(1, 1, 1)~(9, 1, 1)
~(2, 1, 1)~(3, 1, 1)
~(2, 1, 1)~(4, 1, 1)
~(2, 1, 1)~(5, 1, 1)
~(2, 1, 1)~(6, 1, 1)
~(2, 1, 1)~(7, 1, 1)
~(2, 1, 1)~(8, 1, 1)
~(2, 1, 1)~(9, 1, 1)
~(3, 1, 1)~(4, 1, 1)
~(3, 1, 1)~(5, 1, 1)
~(3, 1, 1)~(6, 1, 1)
~(3, 1, 1)~(7, 1, 1)
~(3, 1, 1)~(8, 1, 1)
~(3, 1, 1)~(9, 1, 1)
~(4, 1, 1)~(5, 1, 1)
~(4, 1, 1)~(6, 1, 1)
~(4, 1, 1)~(7, 1, 1)
~(4, 1, 1)~(8, 1, 1)
~(4, 1, 1)~(9, 1, 1)
~(5, 1, 1)~(6, 1, 1)
~(5, 1, 1)~(7, 1, 1)
~(5, 1, 1)~(8, 1, 1)
~(5, 1, 1)~(9, 1, 1)
~(6, 1, 1)~(7, 1, 1)
~(6, 1, 1)~(8, 1, 1)
~(6, 1, 1)~(9, 1, 1)
~(7, 1, 1)~(8, 1, 1)
~(7, 1, 1)~(9, 1, 1)
~(8, 1, 1)~(9, 1, 1)
~(1, 1, 2)~(2, 1, 2)
~(1, 1, 2)~(3, 1, 2)
~(1, 1, 2)~(4, 1, 2)
~(1, 1, 2)~(5, 1, 2)
~(1, 1, 2)~(6, 1, 2)
~(1, 1, 2)~(7, 1, 2)
~(1, 1, 2)~(8, 1, 2)
~(1, 1, 2)~(9, 1, 2)
~(2, 1, 2)~(3, 1, 2)
~(2, 1, 2)~(4, 1, 2)
~(2, 1, 2)~(5, 1, 2)
~(2, 1, 2)~(6

~(5, 4, 7)~(8, 4, 7)
~(5, 4, 7)~(9, 4, 7)
~(6, 4, 7)~(7, 4, 7)
~(6, 4, 7)~(8, 4, 7)
~(6, 4, 7)~(9, 4, 7)
~(7, 4, 7)~(8, 4, 7)
~(7, 4, 7)~(9, 4, 7)
~(8, 4, 7)~(9, 4, 7)
~(1, 4, 8)~(2, 4, 8)
~(1, 4, 8)~(3, 4, 8)
~(1, 4, 8)~(4, 4, 8)
~(1, 4, 8)~(5, 4, 8)
~(1, 4, 8)~(6, 4, 8)
~(1, 4, 8)~(7, 4, 8)
~(1, 4, 8)~(8, 4, 8)
~(1, 4, 8)~(9, 4, 8)
~(2, 4, 8)~(3, 4, 8)
~(2, 4, 8)~(4, 4, 8)
~(2, 4, 8)~(5, 4, 8)
~(2, 4, 8)~(6, 4, 8)
~(2, 4, 8)~(7, 4, 8)
~(2, 4, 8)~(8, 4, 8)
~(2, 4, 8)~(9, 4, 8)
~(3, 4, 8)~(4, 4, 8)
~(3, 4, 8)~(5, 4, 8)
~(3, 4, 8)~(6, 4, 8)
~(3, 4, 8)~(7, 4, 8)
~(3, 4, 8)~(8, 4, 8)
~(3, 4, 8)~(9, 4, 8)
~(4, 4, 8)~(5, 4, 8)
~(4, 4, 8)~(6, 4, 8)
~(4, 4, 8)~(7, 4, 8)
~(4, 4, 8)~(8, 4, 8)
~(4, 4, 8)~(9, 4, 8)
~(5, 4, 8)~(6, 4, 8)
~(5, 4, 8)~(7, 4, 8)
~(5, 4, 8)~(8, 4, 8)
~(5, 4, 8)~(9, 4, 8)
~(6, 4, 8)~(7, 4, 8)
~(6, 4, 8)~(8, 4, 8)
~(6, 4, 8)~(9, 4, 8)
~(7, 4, 8)~(8, 4, 8)
~(7, 4, 8)~(9, 4, 8)
~(8, 4, 8)~(9, 4, 8)
~(1, 4, 9)~(2, 4, 9)
~(1, 4, 9)~(3, 4, 9)
~(1, 4, 9)~(4, 4, 9)
~(1, 4, 9)~(5

## 3) For each row, for each number, exactly one varialbe is true.

In [16]:
def atleast_row(names):
    encode = []
    for i in range(n):
        for k in range(n):
            arr = [int(names[i][j][k]) for j in range(n)]
            encode.append(arr)
    return encode

def atmost_row(names):
    encode = []
    for i in range(n):
        for l in range(n):
            for j in range(n-1):
                for k in range(j+1, n):
                    arr = [int(-names[i,j,l]), int(-names[i,k,l])]
                    encode.append(arr)
    return encode

def exactly_one_row(names):
    enc1 = atmost_row(names)
    enc2 = atleast_row(names)
    return enc1 + enc2

pycosat.solve(exactly_one_row(names))

[-1,
 -2,
 -3,
 -4,
 -5,
 -6,
 -7,
 -8,
 -9,
 -10,
 -11,
 -12,
 -13,
 -14,
 -15,
 -16,
 -17,
 -18,
 -19,
 -20,
 -21,
 -22,
 -23,
 -24,
 -25,
 -26,
 -27,
 -28,
 -29,
 -30,
 -31,
 -32,
 -33,
 -34,
 -35,
 -36,
 -37,
 -38,
 -39,
 -40,
 -41,
 -42,
 -43,
 -44,
 -45,
 -46,
 -47,
 -48,
 -49,
 -50,
 -51,
 -52,
 -53,
 -54,
 -55,
 -56,
 -57,
 -58,
 -59,
 -60,
 -61,
 -62,
 -63,
 -64,
 -65,
 -66,
 -67,
 -68,
 -69,
 -70,
 -71,
 -72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 -82,
 -83,
 -84,
 -85,
 -86,
 -87,
 -88,
 -89,
 -90,
 -91,
 -92,
 -93,
 -94,
 -95,
 -96,
 -97,
 -98,
 -99,
 -100,
 -101,
 -102,
 -103,
 -104,
 -105,
 -106,
 -107,
 -108,
 -109,
 -110,
 -111,
 -112,
 -113,
 -114,
 -115,
 -116,
 -117,
 -118,
 -119,
 -120,
 -121,
 -122,
 -123,
 -124,
 -125,
 -126,
 -127,
 -128,
 -129,
 -130,
 -131,
 -132,
 -133,
 -134,
 -135,
 -136,
 -137,
 -138,
 -139,
 -140,
 -141,
 -142,
 -143,
 -144,
 -145,
 -146,
 -147,
 -148,
 -149,
 -150,
 -151,
 -152,
 -153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161

## 4) For each block, for each number, exactly one variable is true.

In [17]:
def atleast_region(names):
    encode = []
    region_size = int(sqrt(n))
    
    for z in range(n):
        for i in range(region_size):
            for j in range(region_size):
                clause = []
                for x in range(region_size):
                    for y in range(region_size):
                        clause.append(int(names[3*i+x][3*j+y][z]))
                encode.append(clause)
    return encode

def atmost_region(names):
    encode = []
    region_size = int(sqrt(n))

    for a in range(n):
        for b in range(region_size - 1):
            for c in range(region_size - 1):
                for d in range(region_size):
                    for e in range(region_size):
                        for f in range(e+1, region_size):
                            literal = int(-names[(region_size*b+e),(region_size*c+e), a])
                            literal2 = int(-names[(region_size*b+e),(region_size*c+f), a])
                            encode.append([literal, literal2])
                        for f in range(d+1, region_size):
                            for g in range(region_size):
                                literal = int(-names[(region_size*b+e),(region_size*c+e), a])
                                literal2 = int(-names[(region_size*b+f),(region_size*c+g), a])
                                encode.append([literal, literal2])
    return encode

def exactly_one_region(names):
    enc1 = atmost_region(names)
    enc2 = atleast_region(names)
    return enc1 + enc2


## 5) Now put them together and see if your solver returns an assignment

In [38]:
encoding = []
encoding.extend(exactly_one(names))
encoding.extend(exactly_one_row(names))
encoding.extend(exactly_one_column(names))
encoding.extend(exactly_one_region(names))

# print(encoding)
pycosat.solve(encoding)

'UNSAT'

In [ ]:
# TODO

## 6) What does this assignment tell you? Can you decode it and print the solution (mind the indices)?